# HuggingFace Tutorial

This is a tutorial for me to learn how to use transformer with huggingface.

# Reference: 
- https://huggingface.co/
- https://huggingface.co/transformers/
- https://github.com/huggingface/datasets
- https://colab.research.google.com/drive/1IPkZo1Wd-DghIOK6gJpcb0Dv4_Gv2kXB?usp=sharing#scrollTo=ImupuGXDGq7b
- https://github.com/monologg/KoELECTRA
- https://github.com/monologg/KoELECTRA/blob/master/finetune/run_squad.py
- Korean Sentence Splitter: https://github.com/hyunwoongko/kss

# Installation

```bash
$ pip install transformers
```

In [1]:
import numpy as np

import torch
import torchtext

print(f"PyTorch Version: {torch.__version__}")
print(f"TorchText Version: {torchtext.__version__}")  

PyTorch Version: 1.6.0
TorchText Version: 0.8.0a0+c851c3e


161306 / 84759

# Datasets

need to install sentencepiece

```bash
$ pip install sentencepiece
$ pip install datasets
```

# How to use?

## Pipeline

- ConversationalPipeline
- FeatureExtractionPipeline
- FillMaskPipeline
- QuestionAnsweringPipeline
- SummarizationPipeline
- TextClassificationPipeline
- TextGenerationPipeline
- TokenClassificationPipeline
- TranslationPipeline
- ZeroShotClassificationPipeline
- Text2TextGenerationPipeline
- TableQuestionAnsweringPipeline

function: `pipeline`

- "feature-extraction": will return a FeatureExtractionPipeline.
- "sentiment-analysis": will return a TextClassificationPipeline.
- "ner": will return a TokenClassificationPipeline.
- "question-answering": will return a QuestionAnsweringPipeline.
- "fill-mask": will return a FillMaskPipeline.
- "summarization": will return a SummarizationPipeline.
- "translation_xx_to_yy": will return a TranslationPipeline.
- "text2text-generation": will return a Text2TextGenerationPipeline.
- "text-generation": will return a TextGenerationPipeline.
- "zero-shot-classification:: will return a ZeroShotClassificationPipeline.
- "conversation": will return a ConversationalPipeline.

model will be automatically downloaded in `~/.cache/huggingface/`

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", framework="pt")

In [ ]:
sentences = [
    "We are very happy to show you the 🤗 Transformers library.",
    "I'll go to Apple Store.",
    "This model covers a lot area. But, I won't use it. Since it is too hard to use."
]

In [ ]:
results = classifier(sentences)
for res in results:
    print(f"label: {res['label']}, with score: {round(res['score'], 4)}")

# Fine-Tuning with Custom Dataset

https://huggingface.co/transformers/custom_datasets.html#qa-squad


## Dataset


- 제목(title)
- 본문의 카테고리(source)
- 본문(context)
- 질문 번호(id)
- 육하원칙(classtype)
- 질문(question)
- 정답의 시작위치(answer_start)
- 정답(text)

In [117]:
import json
from tqdm import tqdm
from pathlib import Path
repo_path = Path().absolute().parent
data_path = repo_path.parent / "data" / "AIhub" / "QA"
for p in data_path.glob("ko_nia*all.json"):
    print(p)

/home/simonjisu/code/data/AIhub/QA/ko_nia_normal_squad_all.json
/home/simonjisu/code/data/AIhub/QA/ko_nia_clue0529_squad_all.json
/home/simonjisu/code/data/AIhub/QA/ko_nia_noanswer_squad_all.json


In [250]:
path = data_path / "ko_nia_clue0529_squad_all_preprocessed.json"
with open(path, 'rb') as f:
    squad_dict = json.load(f)
x = []
for group in tqdm(squad_dict["data"], total=len(squad_dict["data"]), desc="Getting Length of Tokenized Context"):
    for paragraph in group["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            qas_id = qa["id"]
            if qas_id == 'm6_415182-2':
                x.append(group)


Getting Length of Tokenized Context: 100%|██████████| 34500/34500 [00:00<00:00, 765222.04it/s]


In [251]:
x[0]["paragraphs"][0]["qas"]

[{'classtype': 'work_what',
  'id': 'm6_415181-1',
  'answers': [{'text': '스타트업의', 'answer_start': 44}],
  'question': '비비큐가 2017년 부터 무엇의 후원을 이어가고 있어?',
  'clue': [{'clue_start': 66,
    'clue_text': '2018년 첫 스타트업 후원으로 &apos;미드레이트&apos;(P2P/소셜 기반 투자 스타트업)의 워크샵에 참여했다'}]},
 {'classtype': 'work_what',
  'id': 'm6_415181-2',
  'answers': [{'text': '스타트업의', 'answer_start': 44}],
  'question': '비비큐가 2017년 부터 어디의 후원을 이어가고 있대?',
  'clue': [{'clue_start': 66,
    'clue_text': '2018년 첫 스타트업 후원으로 &apos;미드레이트&apos;(P2P/소셜 기반 투자 스타트업)의 워크샵에 참여했다'}]},
 {'classtype': 'work_who',
  'id': 'm6_415182-1',
  'answers': [{'text': '국내 1위 치킨 프랜차이즈 기업 비비큐(회장 윤홍근)는 ', 'answer_start': 0}],
  'question': '누가 2018년 첫 스타트업 후원으로 &apos;미드레이트&apos;(P2P/소셜 기반 투자 스타트업)의 워크샵에 참여했어?',
  'clue': [{'clue_start': 231,
    'clue_text': 'BBQ는 서울대학교 창업동아리 ‘끌림&apos;, 액티비티 예약서비스 ‘데얼&apos;, 이사 스타트업 ‘짐카&apos;, 가치 문화 확산 네트워크 ‘VIRUS&apos;, 반려동물 서비스 ‘페오펫&apos;, 한복 큐레이션샵 ‘하플리&apos; 등 잠재력 있는 스타트업을 숙박서비스 기업 여기어때와 함께 발굴, 지속적인 후원을 이어가고 있다

In [266]:
for path in data_path.glob("ko_nia*all.json"):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)
    missing_context_ids = []
    missing_qas_ids = []
    for i, group in tqdm(enumerate(squad_dict["data"]), total=len(squad_dict["data"]), desc="Reading datset"):
        for paragraph in group["paragraphs"]:
            context = paragraph["context"]
            if context == "":
                missing_context_ids.append(i)
                continue
            for qa in paragraph["qas"]:
                qas_id = qa["id"]
                question_text = qa["question"]
                if question_text == "":
                    missing_qas_ids.append(qas_id)
    for i in range(len(squad_dict["data"])):
        if i in missing_context_ids:
            squad_dict["data"].pop(i)
            continue
        qas = squad_dict["data"][i]["paragraphs"][0]["qas"]
        for j, qa in enumerate(qas):
            if qa["id"] in missing_qas_ids:
                squad_dict["data"][i]["paragraphs"][0]["qas"].pop(j)
                print(f"dropped id: {qa['id']}")
    p = data_path / (path.name.strip(".json") + "_preprocessed.json")
    with open(p, "w") as f:
        json.dump(squad_dict, f)

Reading datset: 100%|██████████| 34500/34500 [00:00<00:00, 482808.46it/s]


dropped id: m6_413880-2
dropped id: m6_415182-2


Reading datset: 100%|██████████| 20030/20030 [00:00<00:00, 483338.18it/s]


In [265]:
for q in qas:
    print(q["id"])

m6_438965-1
m6_438965-2


## to see if we have over 512 tokens of data

In [ ]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering
from tqdm import tqdm

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

def filter_dataset(args_dict):
    data_path = args_dict["data_path"]
    model_name_or_path = args_dict["model_name_or_path"]
    tokenizer = ElectraTokenizer.from_pretrained(model_name_or_path)
    
    length_data = {}
    for path in data_path.glob("ko_nia*all.json"):
        if path.name == "ko_nia_normal_squad_all.json":
            state = "train"
        elif path.name == "ko_nia_clue0529_squad_all.json":
            state = "val"
        else:
            continue
        with open(path, 'rb') as f:
            squad_dict = json.load(f)
        length_of_texts = []
        for group in tqdm(squad_dict["data"], total=len(squad_dict["data"]), desc="Getting Length of Tokenized Context"):
            for paragraph in group["paragraphs"]:
                context = paragraph["context"]
                length_of_texts.append(len(tokenizer.encode(context)))
        length_of_texts = np.array(length_of_texts)
        for token_length in [512, 1024]:
            print(f"Processing: {state}/{token_length}")
            filter_n = token_length if token_length == 512 else token_length*2
            idx_over = np.arange(len(length_of_texts))[length_of_texts >= filter_n]
            print(f"count: # of over {token_length} is {len(idx_over)}/{len(length_of_texts)}, percentage: {len(idx_over)/len(length_of_texts)*100:.2f}%")

            choose_data = []
            for i in tqdm(range(len(squad_dict["data"])), total=len(squad_dict["data"]), desc=f"{state}, {token_length}"):
                if i in idx_over:
                    continue
                else:
                    choose_data.append(squad_dict["data"][i])
            temp_data = dict(
                creator=squad_dict["creator"], 
                version=squad_dict["version"], 
                data=choose_data
            )
            p = data_path / f"AIhub_squad_{state}_{token_length}.json"
            with open(p, "w") as f:
                json.dump(temp_data, f)

In [ ]:
args_dict = {
    "data_path": data_path
    "model_name_or_path": "monologg/koelectra-base-v3-discriminator"
}
filter_dataset(args_dict)

## split dataset

In [ ]:
n_split = 10
processd_length = []
for path in data_path.glob("*all.json"):
    if path.name == "ko_nia_normal_squad_all.json":
        state = "train"
    elif path.name == "ko_nia_clue0529_squad_all.json":
        state = "val"
    else:
        continue
    # read
    with open(path, 'rb') as f:
        squad_dict = json.load(f)
    total_examples = len(squad_dict["data"])
    k = len(squad_dict["data"]) // n_split
    processed = 0
    
    for i in range(n_split):
        p = data_path / f"{state}_{i}.json"
        temp_data = dict(
            creator=squad_dict["creator"], 
            version=squad_dict["version"], 
            data=squad_dict["data"][i:i+k]
        )
        processed += k
        with open(p, "w") as f:
            json.dump(temp_data, f)
            
    if processed < total_examples:
        p = data_path / f"{state}_{i+1}.json"
        temp_data = dict(
            creator=squad_dict["creator"], 
            version=squad_dict["version"], 
            data=squad_dict["data"][processed:]
        )
        with open(p, "w") as f:
            json.dump(temp_data, f)

## Explore Datset

In [185]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in tqdm(squad_dict["data"], total=len(squad_dict["data"]), desc="Reading Dataset"):
        for paragraph in group['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

In [186]:
train_file = "ko_nia_normal_squad_all.json"
train_path = data_path / train_file
val_file = "ko_nia_clue0529_squad_all.json"
val_path = data_path / val_file

train_contexts, train_questions, train_answers = read_squad(train_path)
val_contexts, val_questions, val_answers = read_squad(val_path)

Reading Dataset: 100%|██████████| 34500/34500 [00:00<00:00, 457291.48it/s]


In [5]:
print(len(train_contexts), len(val_contexts))

243425 96663


Let's see some samples

In [192]:
import termcolor

for idx in np.random.randint(0, len(val_contexts), size=(2,)):
    txt = train_answers[idx]["text"]
    context = train_contexts[idx].split(txt)
    context.insert(1, termcolor.colored(txt, "red", attrs=["bold"]))
    answer_end = train_answers[idx]['answer_start'] + len(train_answers[idx]['text'])  # not included like python range
    print(termcolor.colored(f"[{idx}]Context: ", attrs=["bold"]))
    print("".join(context))
    print(termcolor.colored(f"[{idx}]Question: ", attrs=["bold"]))
    print(train_questions[idx])
    print(termcolor.colored(f"[{idx}]Answer: ", attrs=["bold"]))
    print(f"  Start: {train_answers[idx]['answer_start']}, End: {answer_end}")
    print()

[58567]Context: 
‘미스티’(JTBC) ‘리턴’(SBS) ‘작은 신의 아이들’(OCN) ‘마더’(tvN). 최근 방송된 이 드라마들의 공통점은 세 가지다. 미스터리를 기반으로 스토리가 전개된다는 것, 탄탄한 이야기 구조로 호평을 받았다는 것, 극본을 쓴 작가들의 첫 장편 드라마라는 것이다. ‘미스티’를 쓴 제인(본명 김재인), ‘리턴’의 최경미, ‘작은 신의 아이들’ 한우리, ‘마더’ 정서경. 네 작가 모두 신인이다. 개연성이 떨어지거나 속도 조절을 못 하면 금세 허술해지기 쉬운 미스터리를 차용해 작품을 써 냈다. 짜임새 있는 구성과 신선한 전개로 방송가에 새로운 활력을 불어넣고 있다. 드라마는 감독의 연출, 배우의 연기도 중요하지만 작가의 필력이 작품의 완성도를 최종적으로 결정한다. 드라마를 ‘작가의 예술’이라고 부르는 건 그래서다. 작가의 실력에 따라 작품 수준이 좌우된다. 이들이 장편 드라마 경험이 없는데도 완성도 높은 작품을 내놓을 수 있었던 비결은 뭘까. 작가들의 경력을 보면 어느 정도 납득이 간다. 제인 작가는 교양 방송 부문에서 20년 가까이 일해 온 것으로 알려졌고, 한우리 작가는 시사 프로그램 ‘그것이 알고 싶다’(SBS) 작가 출신이다. 최경미 작가는 2007년 단막극 ‘아귀’(KBS)를 썼고 2015년 SBS문화재단 극본 공모에서 최우수상을 받으며 두각을 나타냈다. 정서경 작가는 충무로에서 먼저 인정받았다. 한국예술종합학교 영상원 출신으로 영화 ‘친절한 금자씨’(2005), ‘싸이보그지만 괜찮아’(2006), ‘박쥐’(2009), ‘아가씨’(2016) 등의 각본을 썼다. 네 사람 모두 각자의 분야에서 작가로서 경력을 차근차근 쌓아온 셈이다. 명성 대신 작품성을 선택한 방송사의 안목도 중요하게 작용했다. 신인 작가들이 활약한 방송사들을 보면 케이블이나 종합편성채널이 많다. 지상파 방송이 시청률에 급급해 공식화된 드라마를 답습하는데 반해 케이블이나 종편은 시청률 성적에 덜 얽매이다보니 새로운 시도를 주저하지 않는다. 이런 상황이 계

In [190]:
def add_end_idx(answers, contexts):
    for idx, (answer, context) in enumerate(zip(answers, contexts)):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
            print(f"type1: {idx}")
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
            print(f"type2: {idx}")

In [8]:
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [9]:
from transformers import ElectraModel, ElectraTokenizer, ElectraTokenizerFast, ElectraForQuestionAnswering

tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")  
# Fast 를 써야 ._encodings 속성이 생긴다. 
# 안에는 Encoding class로 된 데이터가 list롤 있음
# train_encodings._encodings[0]
# Encoding(num_tokens=365, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [10]:
sample_len = 5
train_encodings = tokenizer(
    train_contexts[:sample_len], train_questions[:sample_len], padding = "max_length",
    max_length=512, truncation=True
)
val_encodings = tokenizer(
    val_contexts[:sample_len], val_questions[:sample_len], padding = "max_length",
    max_length=512, truncation=True
)

In [11]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))  # char_to_token: 문자가 몇 번째 토큰에 있는지 확인
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers[:sample_len])
add_token_positions(val_encodings, val_answers[:sample_len])

In [12]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [13]:
class ARGS:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
args_dict = {
  "task": "korquad",
  "data_dir": "data",
  "ckpt_dir": "ckpt",
  "train_file": "KorQuAD_v1.0_train.json",
  "predict_file": "KorQuAD_v1.0_dev.json",
  "threads": 4,
  "version_2_with_negative": False,
  "null_score_diff_threshold": 0.0,
  "max_seq_length": 512,
  "doc_stride": 128,
  "max_query_length": 64,
  "max_answer_length": 30,
  "n_best_size": 20,
  "verbose_logging": True,
  "overwrite_output_dir": True,
  "evaluate_during_training": True,
  "eval_all_checkpoints": True,
  "save_optimizer": False,
  "do_lower_case": False,
  "do_train": True,
  "do_eval": True,
  "num_train_epochs": 7,
  "weight_decay": 0.0,
  "gradient_accumulation_steps": 1,
  "adam_epsilon": 1e-8,
  "warmup_proportion": 0,
  "max_steps": -1,
  "max_grad_norm": 1.0,
  "no_cuda": False,
  "model_type": "koelectra-base-v3",
  "model_name_or_path": "monologg/koelectra-base-v3-discriminator",
  "output_dir": "koelectra-base-v3-korquad-ckpt",
  "seed": 42,
  "train_batch_size": 8,
  "eval_batch_size": 32,
  "logging_steps": 1000,
  "save_steps": 1000,
  "learning_rate": 5e-5
}
     
args = ARGS(**args_dict)

In [14]:
from transformers import ElectraForQuestionAnswering, ElectraConfig
config = ElectraConfig.from_pretrained(args.model_name_or_path)
model = ElectraForQuestionAnswering.from_pretrained(args.model_name_or_path, config=config)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.weight'

In [15]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = "cpu"

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=args.train_batch_size, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for batch in train_loader:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()
    break

In [16]:
print("Inputs", input_ids.size())
print("Start Tokens", start_positions.size())
print("End Tokens", end_positions.size())
print("Loss", outputs.loss)
print("Start Logits", outputs.start_logits.size())
print("End Logits", outputs.end_logits.size())

Inputs torch.Size([5, 512])
Start Tokens torch.Size([5])
End Tokens torch.Size([5])
Loss tensor(6.0529, grad_fn=<DivBackward0>)
Start Logits torch.Size([5, 512])
End Logits torch.Size([5, 512])


## Is there more Efficient way???...

In [6]:
from transformers import squad_convert_examples_to_features
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor

In [267]:
processor = SquadV2Processor()
examples = processor.get_train_examples(data_dir=data_path, filename="ko_nia_normal_squad_all.json")  # examples은 먼저 whitespace 기반으로 토크나이징함

100%|██████████| 34500/34500 [00:31<00:00, 1088.61it/s]


In [13]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [6]:
features, train_dataset = squad_convert_examples_to_features(
    examples=examples,
    tokenizer=tokenizer,
    max_seq_length=512,
    doc_stride=128,
    max_query_length=64,
    is_training=True,
    return_dataset="pt",
    threads=4,
)

add example index and unique id: 100%|██████████| 25/25 [00:00<00:00, 67216.41it/s]


In [7]:
a = features[1]
print(a.start_position, a.end_position)

26 30


In [8]:
for i, t in enumerate(a.tokens[:35]):
    print(i, t)

0 [CLS]
1 '
2 국제
3 청소년
4 ##포
5 ##럼
6 '
7 이
8 열리
9 ##는
10 때
11 ##는
12 ?
13 [SEP]
14 한국
15 ##청
16 ##소년단
17 ##체
18 ##협
19 ##의
20 ##회
21 ##와
22 여성
23 ##가족
24 ##부
25 ##는
26 22
27 ##일
28 ##부터
29 28
30 ##일
31 ##까
32 ##지
33 서울
34 ##과


In [178]:
class ARGS:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
args_dict = {
  "task": "korquad",
  "data_dir": "data",
  "ckpt_dir": "ckpt",
  "train_file": "KorQuAD_v1.0_train.json",
  "predict_file": "KorQuAD_v1.0_dev.json",
  "threads": 4,
  "version_2_with_negative": False,
  "null_score_diff_threshold": 0.0,
  "max_seq_length": 512,
  "doc_stride": 128,
  "max_query_length": 64,
  "max_answer_length": 30,
  "n_best_size": 20,
  "verbose_logging": True,
  "overwrite_output_dir": True,
  "evaluate_during_training": True,
  "eval_all_checkpoints": True,
  "save_optimizer": False,
  "do_lower_case": False,
  "do_train": True,
  "do_eval": True,
  "num_train_epochs": 7,
  "weight_decay": 0.0,
  "gradient_accumulation_steps": 1,
  "adam_epsilon": 1e-8,
  "warmup_proportion": 0,
  "max_steps": -1,
  "max_grad_norm": 1.0,
  "no_cuda": False,
  "model_type": "koelectra-base-v3",
  "model_name_or_path": "monologg/koelectra-base-v3-discriminator",
  "output_dir": "koelectra-base-v3-korquad-ckpt",
  "seed": 42,
  "train_batch_size": 8,
  "eval_batch_size": 32,
  "logging_steps": 1000,
  "save_steps": 1000,
  "learning_rate": 5e-5
}
     
args = ARGS(**args_dict)

def tolist(tensor):
    return tensor.detach().cpu().tolist()

In [179]:
from transformers import ElectraForQuestionAnswering, ElectraConfig
config = ElectraConfig.from_pretrained(args.model_name_or_path)
model = ElectraForQuestionAnswering.from_pretrained(args.model_name_or_path, config=config)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.weight'

In [180]:
device = "cpu"

In [12]:
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=args.train_batch_size)
t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=int(t_total * args.warmup_proportion), num_training_steps=t_total
)

global_step = 1
epochs_trained = 0
steps_trained_in_current_epoch = 0
tr_loss, logging_loss = 0.0, 0.0
model.zero_grad()

In [13]:
model.to(device)
model.train()
for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)

    inputs = {
        "input_ids": batch[0],
        "attention_mask": batch[1],
        "token_type_ids": batch[2],
        "start_positions": batch[3],
        "end_positions": batch[4],
    }
    break

In [14]:
input_ids=inputs["input_ids"]
attention_mask=inputs["attention_mask"]
token_type_ids=inputs["token_type_ids"]

In [15]:
o = model.electra.forward(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
o.last_hidden_state.size()

torch.Size([8, 512, 768])

In [16]:
fin_o = model.qa_outputs(o.last_hidden_state)
fin_o.size()

torch.Size([8, 512, 2])

In [17]:
outputs = model(**inputs)
print(f"start predict {outputs.start_logits.argmax(1)}")
print(f"start answer {inputs['start_positions']}")
print(f"start predict {outputs.end_logits.argmax(1)}")
print(f"start answer {inputs['end_positions']}")

start predict tensor([257,  92,  26, 461,  28,  28, 468,  86])
start answer tensor([  0, 108,  21,  87,  18,  16, 114,  37])
start predict tensor([243,  64,  56, 178, 109, 109,  21, 333])
start answer tensor([  0, 119,  31,  91,  18,  16, 116,  43])


**eval phase**

In [47]:
from transformers.tokenization_utils_base import TruncationStrategy

def _new_check_is_max_context(doc_spans, cur_span_index, position):
    """Check if this is the 'max context' doc span for the token."""
    # if len(doc_spans) == 1:
    # return True
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span["start"] + doc_span["length"] - 1
        if position < doc_span["start"]:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span["start"]
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span["length"]
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index

In [48]:
from transformers import SquadFeatures, BatchEncoding, PreTrainedTokenizerBase
MULTI_SEP_TOKENS_TOKENIZERS_SET = {"roberta", "camembert", "bart", "mpnet"}

def squad_convert_example_to_features(
    example, max_seq_length, doc_stride, max_query_length, padding_strategy, is_training
):
    features = []
    if is_training and not example.is_impossible:
        # Get start and end position
        start_position = example.start_position
        end_position = example.end_position

        # If the answer cannot be found in the text, then skip this example.
        actual_text = " ".join(example.doc_tokens[start_position : (end_position + 1)])
        cleaned_answer_text = " ".join(whitespace_tokenize(example.answer_text))
        if actual_text.find(cleaned_answer_text) == -1:
            logger.warning("Could not find answer: '%s' vs. '%s'", actual_text, cleaned_answer_text)
            return []

    tok_to_orig_index = []
    orig_to_tok_index = []
    all_doc_tokens = []
    for (i, token) in enumerate(example.doc_tokens):
        orig_to_tok_index.append(len(all_doc_tokens))
        if tokenizer.__class__.__name__ in [
            "RobertaTokenizer",
            "LongformerTokenizer",
            "BartTokenizer",
            "RobertaTokenizerFast",
            "LongformerTokenizerFast",
            "BartTokenizerFast",
        ]:
            sub_tokens = tokenizer.tokenize(token, add_prefix_space=True)
        else:
            sub_tokens = tokenizer.tokenize(token)
        for sub_token in sub_tokens:
            tok_to_orig_index.append(i)
            all_doc_tokens.append(sub_token)

    if is_training and not example.is_impossible:
        tok_start_position = orig_to_tok_index[example.start_position]
        if example.end_position < len(example.doc_tokens) - 1:
            tok_end_position = orig_to_tok_index[example.end_position + 1] - 1
        else:
            tok_end_position = len(all_doc_tokens) - 1

        (tok_start_position, tok_end_position) = _improve_answer_span(
            all_doc_tokens, tok_start_position, tok_end_position, tokenizer, example.answer_text
        )

    spans = []

    truncated_query = tokenizer.encode(
        example.question_text, add_special_tokens=False, truncation=True, max_length=max_query_length
    )

    # Tokenizers who insert 2 SEP tokens in-between <context> & <question> need to have special handling
    # in the way they compute mask of added tokens.
    tokenizer_type = type(tokenizer).__name__.replace("Tokenizer", "").lower()
    sequence_added_tokens = (
        tokenizer.model_max_length - tokenizer.max_len_single_sentence + 1
        if tokenizer_type in MULTI_SEP_TOKENS_TOKENIZERS_SET
        else tokenizer.model_max_length - tokenizer.max_len_single_sentence
    )
    sequence_pair_added_tokens = tokenizer.model_max_length - tokenizer.max_len_sentences_pair

    span_doc_tokens = all_doc_tokens
    while len(spans) * doc_stride < len(all_doc_tokens):

        # Define the side we want to truncate / pad and the text/pair sorting
        if tokenizer.padding_side == "right":
            texts = truncated_query
            pairs = span_doc_tokens
            truncation = TruncationStrategy.ONLY_SECOND.value
        else:
            texts = span_doc_tokens
            pairs = truncated_query
            truncation = TruncationStrategy.ONLY_FIRST.value

        encoded_dict = tokenizer.encode_plus(  # TODO(thom) update this logic
            texts,
            pairs,
            truncation=truncation,
            padding=padding_strategy,
            max_length=max_seq_length,
            return_overflowing_tokens=True,
            stride=max_seq_length - doc_stride - len(truncated_query) - sequence_pair_added_tokens,
            return_token_type_ids=True,
        )

        paragraph_len = min(
            len(all_doc_tokens) - len(spans) * doc_stride,
            max_seq_length - len(truncated_query) - sequence_pair_added_tokens,
        )

        if tokenizer.pad_token_id in encoded_dict["input_ids"]:
            if tokenizer.padding_side == "right":
                non_padded_ids = encoded_dict["input_ids"][: encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
            else:
                last_padding_id_position = (
                    len(encoded_dict["input_ids"]) - 1 - encoded_dict["input_ids"][::-1].index(tokenizer.pad_token_id)
                )
                non_padded_ids = encoded_dict["input_ids"][last_padding_id_position + 1 :]

        else:
            non_padded_ids = encoded_dict["input_ids"]

        tokens = tokenizer.convert_ids_to_tokens(non_padded_ids)

        token_to_orig_map = {}
        for i in range(paragraph_len):
            index = len(truncated_query) + sequence_added_tokens + i if tokenizer.padding_side == "right" else i
            token_to_orig_map[index] = tok_to_orig_index[len(spans) * doc_stride + i]

        encoded_dict["paragraph_len"] = paragraph_len
        encoded_dict["tokens"] = tokens
        encoded_dict["token_to_orig_map"] = token_to_orig_map
        encoded_dict["truncated_query_with_special_tokens_length"] = len(truncated_query) + sequence_added_tokens
        encoded_dict["token_is_max_context"] = {}
        encoded_dict["start"] = len(spans) * doc_stride
        encoded_dict["length"] = paragraph_len

        spans.append(encoded_dict)

        if "overflowing_tokens" not in encoded_dict or (
            "overflowing_tokens" in encoded_dict and len(encoded_dict["overflowing_tokens"]) == 0
        ):
            break
        span_doc_tokens = encoded_dict["overflowing_tokens"]

    for doc_span_index in range(len(spans)):
        for j in range(spans[doc_span_index]["paragraph_len"]):
            is_max_context = _new_check_is_max_context(spans, doc_span_index, doc_span_index * doc_stride + j)
            index = (
                j
                if tokenizer.padding_side == "left"
                else spans[doc_span_index]["truncated_query_with_special_tokens_length"] + j
            )
            spans[doc_span_index]["token_is_max_context"][index] = is_max_context

    for span in spans:
        # Identify the position of the CLS token
        cls_index = span["input_ids"].index(tokenizer.cls_token_id)

        # p_mask: mask with 1 for token than cannot be in the answer (0 for token which can be in an answer)
        # Original TF implem also keep the classification token (set to 0)
        p_mask = np.ones_like(span["token_type_ids"])
        if tokenizer.padding_side == "right":
            p_mask[len(truncated_query) + sequence_added_tokens :] = 0
        else:
            p_mask[-len(span["tokens"]) : -(len(truncated_query) + sequence_added_tokens)] = 0

        pad_token_indices = np.where(span["input_ids"] == tokenizer.pad_token_id)
        special_token_indices = np.asarray(
            tokenizer.get_special_tokens_mask(span["input_ids"], already_has_special_tokens=True)
        ).nonzero()

        p_mask[pad_token_indices] = 1
        p_mask[special_token_indices] = 1

        # Set the cls index to 0: the CLS index can be used for impossible answers
        p_mask[cls_index] = 0

        span_is_impossible = example.is_impossible
        start_position = 0
        end_position = 0
        if is_training and not span_is_impossible:
            # For training, if our document chunk does not contain an annotation
            # we throw it out, since there is nothing to predict.
            doc_start = span["start"]
            doc_end = span["start"] + span["length"] - 1
            out_of_span = False

            if not (tok_start_position >= doc_start and tok_end_position <= doc_end):
                out_of_span = True

            if out_of_span:
                start_position = cls_index
                end_position = cls_index
                span_is_impossible = True
            else:
                if tokenizer.padding_side == "left":
                    doc_offset = 0
                else:
                    doc_offset = len(truncated_query) + sequence_added_tokens

                start_position = tok_start_position - doc_start + doc_offset
                end_position = tok_end_position - doc_start + doc_offset

        features.append(
            SquadFeatures(
                span["input_ids"],
                span["attention_mask"],
                span["token_type_ids"],
                cls_index,
                p_mask.tolist(),
                example_index=0,  # Can not set unique_id and example_index here. They will be set after multiple processing.
                unique_id=0,
                paragraph_len=span["paragraph_len"],
                token_is_max_context=span["token_is_max_context"],
                tokens=span["tokens"],
                token_to_orig_map=span["token_to_orig_map"],
                start_position=start_position,
                end_position=end_position,
                is_impossible=span_is_impossible,
                qas_id=example.qas_id,
            )
        )
    return features

In [268]:
eval_examples = processor.get_dev_examples(data_dir=data_path, filename="ko_nia_clue0529_squad_all_preprocessed.json")

100%|██████████| 34500/34500 [00:28<00:00, 1216.19it/s]


In [269]:
max_seq_length=512
doc_stride=128
max_query_length=64
padding_strategy="max_length"
is_training=False
for i, ex in tqdm(enumerate(eval_examples), total=len(eval_examples)):
    try:
        f = squad_convert_example_to_features(ex, max_seq_length, doc_stride, max_query_length, padding_strategy, is_training)
    except:
        break

 39%|███▉      | 37748/96661 [07:43<12:02, 81.53it/s]  


In [270]:
for a in ex

SyntaxError: invalid syntax (<ipython-input-270-91a9a5cc3ac0>, line 1)

In [272]:
ex.question_text

' '

In [210]:
example=ex


In [238]:

features = []
if is_training and not example.is_impossible:
    # Get start and end position
    start_position = example.start_position
    end_position = example.end_position

    # If the answer cannot be found in the text, then skip this example.
    actual_text = " ".join(example.doc_tokens[start_position : (end_position + 1)])
    cleaned_answer_text = " ".join(whitespace_tokenize(example.answer_text))
    if actual_text.find(cleaned_answer_text) == -1:
        logger.warning("Could not find answer: '%s' vs. '%s'", actual_text, cleaned_answer_text)

tok_to_orig_index = []
orig_to_tok_index = []
all_doc_tokens = []
for (i, token) in enumerate(example.doc_tokens):
    orig_to_tok_index.append(len(all_doc_tokens))
    if tokenizer.__class__.__name__ in [
        "RobertaTokenizer",
        "LongformerTokenizer",
        "BartTokenizer",
        "RobertaTokenizerFast",
        "LongformerTokenizerFast",
        "BartTokenizerFast",
    ]:
        sub_tokens = tokenizer.tokenize(token, add_prefix_space=True)
    else:
        sub_tokens = tokenizer.tokenize(token)
    for sub_token in sub_tokens:
        tok_to_orig_index.append(i)
        all_doc_tokens.append(sub_token)

if is_training and not example.is_impossible:
    tok_start_position = orig_to_tok_index[example.start_position]
    if example.end_position < len(example.doc_tokens) - 1:
        tok_end_position = orig_to_tok_index[example.end_position + 1] - 1
    else:
        tok_end_position = len(all_doc_tokens) - 1

    (tok_start_position, tok_end_position) = _improve_answer_span(
        all_doc_tokens, tok_start_position, tok_end_position, tokenizer, example.answer_text
    )

spans = []

truncated_query = tokenizer.encode(
    example.question_text, add_special_tokens=False, truncation=True, max_length=max_query_length
)

# Tokenizers who insert 2 SEP tokens in-between <context> & <question> need to have special handling
# in the way they compute mask of added tokens.
tokenizer_type = type(tokenizer).__name__.replace("Tokenizer", "").lower()
sequence_added_tokens = (
    tokenizer.model_max_length - tokenizer.max_len_single_sentence + 1
    if tokenizer_type in MULTI_SEP_TOKENS_TOKENIZERS_SET
    else tokenizer.model_max_length - tokenizer.max_len_single_sentence
)
sequence_pair_added_tokens = tokenizer.model_max_length - tokenizer.max_len_sentences_pair

span_doc_tokens = all_doc_tokens
while len(spans) * doc_stride < len(all_doc_tokens):

    # Define the side we want to truncate / pad and the text/pair sorting
    if tokenizer.padding_side == "right":
        texts = truncated_query
        pairs = span_doc_tokens
        truncation = TruncationStrategy.ONLY_SECOND.value
    else:
        texts = span_doc_tokens
        pairs = truncated_query
        truncation = TruncationStrategy.ONLY_FIRST.value

    encoded_dict = tokenizer.encode_plus(  # TODO(thom) update this logic
        texts,
        pairs,
        truncation=truncation,
        padding=padding_strategy,
        max_length=max_seq_length,
        return_overflowing_tokens=True,
        stride=max_seq_length - doc_stride - len(truncated_query) - sequence_pair_added_tokens,
        return_token_type_ids=True,
    )

    paragraph_len = min(
        len(all_doc_tokens) - len(spans) * doc_stride,
        max_seq_length - len(truncated_query) - sequence_pair_added_tokens,
    )

    if tokenizer.pad_token_id in encoded_dict["input_ids"]:
        if tokenizer.padding_side == "right":
            non_padded_ids = encoded_dict["input_ids"][: encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
        else:
            last_padding_id_position = (
                len(encoded_dict["input_ids"]) - 1 - encoded_dict["input_ids"][::-1].index(tokenizer.pad_token_id)
            )
            non_padded_ids = encoded_dict["input_ids"][last_padding_id_position + 1 :]

    else:
        non_padded_ids = encoded_dict["input_ids"]

    tokens = tokenizer.convert_ids_to_tokens(non_padded_ids)

    token_to_orig_map = {}
    for i in range(paragraph_len):
        index = len(truncated_query) + sequence_added_tokens + i if tokenizer.padding_side == "right" else i
        token_to_orig_map[index] = tok_to_orig_index[len(spans) * doc_stride + i]

    encoded_dict["paragraph_len"] = paragraph_len
    encoded_dict["tokens"] = tokens
    encoded_dict["token_to_orig_map"] = token_to_orig_map
    encoded_dict["truncated_query_with_special_tokens_length"] = len(truncated_query) + sequence_added_tokens
    encoded_dict["token_is_max_context"] = {}
    encoded_dict["start"] = len(spans) * doc_stride
    encoded_dict["length"] = paragraph_len

    spans.append(encoded_dict)

    if "overflowing_tokens" not in encoded_dict or (
        "overflowing_tokens" in encoded_dict and len(encoded_dict["overflowing_tokens"]) == 0
    ):
        break
    span_doc_tokens = encoded_dict["overflowing_tokens"]

ValueError: Input [] is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [240]:
tokenizer.encode(
    example.question_text, add_special_tokens=False, truncation=True, max_length=max_query_length
)

[]

In [214]:
encoded_dict = tokenizer.encode_plus(  # TODO(thom) update this logic
    texts,
    pairs,
    truncation=truncation,
    padding=padding_strategy,
    max_length=max_seq_length,
    return_overflowing_tokens=True,
    stride=max_seq_length - doc_stride - len(truncated_query) - sequence_pair_added_tokens,
    return_token_type_ids=True,
)

ValueError: Input [] is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [208]:
squad_convert_example_to_features(ex, 512, 128, 64, "max_length", False)

ValueError: Input [] is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [179]:
for e in examples:
    if "‘" in e.context_text:
        break

---

In [ ]:
eval_features, eval_dataset = squad_convert_examples_to_features(
    examples=eval_examples,
    tokenizer=tokenizer,
    max_seq_length=512,
    doc_stride=128,
    max_query_length=64,
    is_training=False,
    return_dataset="pt",
    threads=4,
)

In [189]:
# for fea in eval_features:
#     fea.unique_id -= 1000000000
eval_dataloader = DataLoader(eval_dataset, shuffle=False, batch_size=args.train_batch_size)

In [37]:
from copy import deepcopy

In [43]:
features = deepcopy(eval_features)
example_index = 0
unique_id = 1000000000
previous_example_index = -1 
for fea in features:
    print(f"before: {fea.unique_id} / {fea.example_index}")
    fea.unique_id = unique_id
    unique_id += 1
    
    current_example_index = fea.example_index
    print(f" c: {current_example_index} / p: {previous_example_index} / e: {example_index}")
    if previous_example_index == current_example_index:
        fea.example_index = previous_example_index
    else:
        previous_example_index = fea.example_index
        fea.example_index = example_index
        example_index += 1
    print(f"after: {fea.unique_id} / {fea.example_index}")
    print()

before: 1000000000 / 0
 c: 0 / p: -1 / e: 0
after: 1000000000 / 0

before: 1000000001 / 1
 c: 1 / p: 0 / e: 1
after: 1000000001 / 1

before: 1000000002 / 2
 c: 2 / p: 1 / e: 2
after: 1000000002 / 2

before: 1000000003 / 3
 c: 3 / p: 2 / e: 3
after: 1000000003 / 3

before: 1000000004 / 4
 c: 4 / p: 3 / e: 4
after: 1000000004 / 4

before: 1000000005 / 5
 c: 5 / p: 4 / e: 5
after: 1000000005 / 5

before: 1000000006 / 5
 c: 5 / p: 5 / e: 6
after: 1000000006 / 5

before: 1000000007 / 6
 c: 6 / p: 5 / e: 6
after: 1000000007 / 6

before: 1000000008 / 6
 c: 6 / p: 6 / e: 7
after: 1000000008 / 6

before: 1000000009 / 7
 c: 7 / p: 6 / e: 7
after: 1000000009 / 7

before: 1000000010 / 7
 c: 7 / p: 7 / e: 8
after: 1000000010 / 7

before: 1000000011 / 8
 c: 8 / p: 7 / e: 8
after: 1000000011 / 8

before: 1000000012 / 8
 c: 8 / p: 8 / e: 9
after: 1000000012 / 8

before: 1000000013 / 9
 c: 9 / p: 8 / e: 9
after: 1000000013 / 9

before: 1000000014 / 9
 c: 9 / p: 9 / e: 10
after: 1000000014 / 9

before: 

In [44]:
features2 = deepcopy(eval_features)

previous_example_index = -1
for fea in features2:
    print(f"before: {fea.unique_id} / {fea.example_index}")
    fea.unique_id = unique_id
    unique_id += 1
    
    current_example_index = fea.example_index
    print(f" c: {current_example_index} / p: {previous_example_index} / e: {example_index}")
    if previous_example_index == current_example_index:
        fea.example_index = previous_example_index
    else:
        previous_example_index = fea.example_index
        fea.example_index = example_index
        example_index += 1
    print(f"after: {fea.unique_id} / {fea.example_index}")
    print()

before: 1000000000 / 0
 c: 0 / p: -1 / e: 25
after: 1000000032 / 25

before: 1000000001 / 1
 c: 1 / p: 0 / e: 26
after: 1000000033 / 26

before: 1000000002 / 2
 c: 2 / p: 1 / e: 27
after: 1000000034 / 27

before: 1000000003 / 3
 c: 3 / p: 2 / e: 28
after: 1000000035 / 28

before: 1000000004 / 4
 c: 4 / p: 3 / e: 29
after: 1000000036 / 29

before: 1000000005 / 5
 c: 5 / p: 4 / e: 30
after: 1000000037 / 30

before: 1000000006 / 5
 c: 5 / p: 5 / e: 31
after: 1000000038 / 5

before: 1000000007 / 6
 c: 6 / p: 5 / e: 31
after: 1000000039 / 31

before: 1000000008 / 6
 c: 6 / p: 6 / e: 32
after: 1000000040 / 6

before: 1000000009 / 7
 c: 7 / p: 6 / e: 32
after: 1000000041 / 32

before: 1000000010 / 7
 c: 7 / p: 7 / e: 33
after: 1000000042 / 7

before: 1000000011 / 8
 c: 8 / p: 7 / e: 33
after: 1000000043 / 33

before: 1000000012 / 8
 c: 8 / p: 8 / e: 34
after: 1000000044 / 8

before: 1000000013 / 9
 c: 9 / p: 8 / e: 34
after: 1000000045 / 34

before: 1000000014 / 9
 c: 9 / p: 9 / e: 35
after: 

In [35]:
for fea in features:
    print(f"previous: {fea.unique_id} / {fea.example_index}")

previous: 1000000000 / 0
previous: 1000000001 / 1
previous: 1000000002 / 1
previous: 1000000003 / 1
previous: 1000000004 / 1
previous: 1000000005 / 1
previous: 1000000006 / 2
previous: 1000000007 / 3
previous: 1000000008 / 4
previous: 1000000009 / 5
previous: 1000000010 / 5
previous: 1000000011 / 6
previous: 1000000012 / 6
previous: 1000000013 / 7
previous: 1000000014 / 7
previous: 1000000015 / 8
previous: 1000000016 / 8
previous: 1000000017 / 9
previous: 1000000018 / 9
previous: 1000000019 / 10
previous: 1000000020 / 10
previous: 1000000021 / 11
previous: 1000000022 / 11
previous: 1000000023 / 12
previous: 1000000024 / 13
previous: 1000000025 / 14
previous: 1000000026 / 15
previous: 1000000027 / 16
previous: 1000000028 / 17
previous: 1000000029 / 18
previous: 1000000030 / 19
previous: 1000000031 / 20


In [190]:
all_results = []
for batch in eval_dataloader:
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
        }
        example_indices = batch[3]
        outputs = model(**inputs)
        
    for i, example_index in enumerate(example_indices):
        eval_feature = eval_features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        output = [tolist(o[i]) for o in outputs.values()]
        start_logits, end_logits = output
        result = SquadResult(unique_id, start_logits, end_logits)
        all_results.append(result)

In [33]:
from transformers.data.metrics.squad_metrics import (
    compute_predictions_logits,
    squad_evaluate
)

In [34]:
output_prediction_file = "./predictions.json"
output_nbest_file = "./nbest_predictions.json"
output_null_log_odds_file = "./null_odds.json"

In [35]:
predictions = compute_predictions_logits(
    eval_examples,
    eval_features,
    all_results,
    args.n_best_size,
    args.max_answer_length,
    args.do_lower_case,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    args.verbose_logging,
    args.version_2_with_negative,
    args.null_score_diff_threshold,
    tokenizer,
)

In [36]:
for i, j in zip(eval_examples, predictions.items()):
    if i.qas_id == j[0]:
        print(i.qas_id)
        print(f"Answer: {i.answers[0]['text']}")
        print(f"Predict: {j[1]}")
        print()
    else:
        print(i.qas_id, j[0])

c1_57059-1
Answer: 한국청소년단체협의회와 여성가족부
Predict: 선언문을 전달할 예정이다

c1_57060-1
Answer: 22일부터 28일
Predict: 선언문을 전달할 예정이다

c1_57061-1
Answer: '청소년과 뉴미디어'
Predict: 선언문을 전달할 예정이다

c1_57062-1
Answer: 기조강연을 시작으로 국가별 주제관련 사례발표, 그룹 토론 및 전체총회, '청소년선언문' 작성 및 채택 등 다양한 프로그램을 운영한다.
Predict: 선언문을 전달할 예정이다

m5_306705-1
Answer: 샐리
Predict: 육상 볼링 양궁 리듬체조 에어로빅 선수권대회(이하 아육대)'에서는

c1_151305-1
Answer: 보조 교통 경찰로 일하는 천중핑
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151306-1
Answer: 지난달 28일
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151307-1
Answer: 구이저우성 카일리시
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151308-1
Answer: ‘중국의 좋은 이웃상’과 함께 상금 1만 위안(약 170만원)을 수여
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151309-1
Answer: 이틀 간의 코마 상태 이후 의식을 회복해 지난 2일부터 중환자실에서 치료를 받고 있습니다
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151310-1
Answer: 열쇠공이 문을 따는 소리에 겁을 먹고 창문 밖으로 도망을 치려다
Predict: 회복해 지난

c1_151311-1
Answer: 아이가 잠든 사이 돌보던 아이의 할머니가 쓰레기를 버리러 나갔다가 문이 잠기는 바람에 열쇠공을 불렀던 것
Predict: 아이는 죽었을 것”이

In [37]:
results = squad_evaluate(eval_examples, predictions)

In [38]:
results

OrderedDict([('exact', 0.0),
             ('f1', 5.556363636363637),
             ('total', 25),
             ('HasAns_exact', 0.0),
             ('HasAns_f1', 5.556363636363637),
             ('HasAns_total', 25),
             ('best_exact', 0.0),
             ('best_exact_thresh', 0.0),
             ('best_f1', 5.556363636363637),
             ('best_f1_thresh', 0.0)])

Why there is 0 to predict?

In [69]:
vocab_rev = {v: k for k, v in tokenizer.vocab.items()}
tostring = lambda x: " ".join(x).replace(" ##", "").replace("[PAD]", "").strip()
def show_original(inputs):
    tokens = [vocab_rev[i.item()] for i in inputs["input_ids"]]
    s, e = inputs["start_positions"].item(), inputs["end_positions"].item()
    print(s, e)
    print("Answer: ", tostring(tokens[s:(e+1)]))
    print(tostring(tokens))

In [78]:
idxes = [5, 6, 7, 8, 9, 10]
for i, idx in enumerate(idxes):
    batch = train_dataset[idx]
    inputs = {
        "input_ids": batch[0],
        "attention_mask": batch[1],
        "token_type_ids": batch[2],
        "start_positions": batch[3],
        "end_positions": batch[4],
    }
    if i % 2 == 0:
        print("----------------------------"*3)
    show_original(inputs)
    print()

------------------------------------------------------------------------------------
103 112
Answer:  보조 교통 경찰로 일하는 천중핑
[CLS] 중국에서 아파트에서 추락하던 3세 아이를 살리고 자신은 혼수상태에 빠진 사람은 누구야 ? [SEP] 중국의 한 여성 경찰이 아파트에서 추락하던 3세 아이를 살리고 자신은 혼수상태에 빠졌습니다 . 의인 ( 義 人 ) 의 소식이 알려지자 각박한 중국 사회에 큰 반향을 일으키고 있습니다 . 5일 귀주도시망 등 중국 현지 언론에 따르면 구이저우성 카일리시에 보조 교통 경찰로 일하는 천중핑 ( 49 ) 은 지난달 28일 한 아파트에서 비상 상황이 발생했다는 연락을 받고 현장으로 향했습니다 . 도착했을 때 아파트 4층 창문에서 여자 아이가 매달려 있었습니다 . 곧이어 아이는 손에 힘이 빠지면서 밑으로 추락했습니다 . 천중핑과 다른 세명의 이웃들이 달려갔습니다 . 그리고 아이는 바닥이 아니라 천중핑의 팔에 떨어졌습니다 . 중간 비막이 천막 때문에 속도가 줄기는 했지만 추락의 충격은 천중핑이 고스란히 감당해야 했습니다 . 아이는 즉시 병원으로 옮겨져 치료를 받았습니다 . 다리 골절로 그리 심각한 상황은 아니라고 합니다 . 하지만 생명의 은인이자 영웅은 커다란 댓가를 치러야 했다 . 뇌출혈로 인한 의식불명 상태에 빠진 것이다 . 다행히 이틀 간의 코마 상태 이후 의식을 회복해 지난 2일부터 중환자실에서 치료를 받고 있습니다 . 아이는 열쇠공이 문을 따는 소리에 겁을 먹고 창문 밖으로 도망을 치려다 사고를 당한 것으로 전해졌습니다 . 아이가 잠든 사이 돌보던 아이의 할머니가 쓰레기를 버리러 나갔다가 문이 잠기는 바람에 열쇠공을 불렀던 것입니다 . 아이의 엄마는 “ 천중핑의 도움이 없었다면 아이는 죽었을 것 ” 이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다 . 카일리시 정부 대표와 공안부 관계자들도 천중핑이 입원한 병원을 찾아 위로하고 회복될때까지 도움

If over 512 tokens, it will generate sentences from the back like: `tokens[-512:]`

In [3]:
from pathlib import Path
import torch
from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl
import torchmetrics
from tqdm import tqdm
from transformers import (
    ElectraForQuestionAnswering, 
    ElectraConfig, 
    ElectraTokenizer,
    AdamW,
    squad_convert_examples_to_features,
    get_linear_schedule_with_warmup
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor
from transformers.data.metrics.squad_metrics import (
    compute_predictions_logits,
    squad_evaluate
)
# typing
from transformers.data.processors import SquadFeatures
from typing import List

In [4]:
train_file = "test_train*.json"
val_file = "test_val*.json"
repo_path = Path().absolute().parent
data_path = repo_path.parent / "data" / "AIhub" / "QA"
ckpt_path = repo_path.parent / "ckpt"

args_dict = {
    "task": "AIHub_QA",
    "data_path": data_path,
    "ckpt_path": ckpt_path,
    "train_file": train_file,
    "val_file": val_file,
    "cache_file": "test_{}_cache_{}",
    "random_seed": 77,
    "threads": 4,
    "version_2_with_negative": False,
    "null_score_diff_threshold": 0.0,
    "max_seq_length": 512,
    "doc_stride": 128,
    "max_query_length": 64,
    "max_answer_length": 30,
    "n_best_size": 20,
    "verbose_logging": True,
    "do_lower_case": False,
    "num_train_epochs": 10,
    "weight_decay": 0.0,
    "adam_epsilon": 1e-8,
    "warmup_proportion": 0,
    "model_type": "koelectra-base-v3",
    "model_name_or_path": "monologg/koelectra-base-v3-discriminator",
    "output_dir": "koelectra-base-v3-korquad-ckpt",
    "seed": 42,
    "train_batch_size": 2,
    "eval_batch_size": 3,
    "learning_rate": 5e-5,
    "output_prediction_file": "predictions/predictions_{}.json",
    "output_nbest_file": "nbest_predictions/nbest_predictions_{}.json",
    "output_null_log_odds_file": "null_odds/null_odds_{}.json",
}

for arg in ["output_prediction_file", "output_nbest_file", "output_null_log_odds_file"]:
    p = args_dict["ckpt_path"] / args_dict[arg]
    if not p.parent.exists():
        p.mkdir(parents=True)

In [5]:

def flatten(li):
    for ele in li:
        if isinstance(ele, list):
            yield from flatten(ele)
        else:
            yield ele

class Model(pl.LightningModule):
    def __init__(self, **kwargs):
        super().__init__()
        self.save_hyperparameters() 
        self.config = ElectraConfig.from_pretrained(self.hparams.model_name_or_path)
        self.model = ElectraForQuestionAnswering.from_pretrained(
            self.hparams.model_name_or_path, 
            config=self.config
        )
        self.tokenizer = ElectraTokenizer.from_pretrained(self.hparams.model_name_or_path)
        # create dataset and cache it
        self.train_files = []
        self.val_files = []
        self.create_dataset_all(state="train")
        self.create_dataset_all(state="val")

        # 
        self.all_examples, self.all_features = [], []

        # function
        self.tolist = lambda x: x.detach().cpu().tolist()

    def create_dataset_all(self, state:str):
        self.example_index = 0
        self.unique_id = 1000000000
        if state == "train":
            file_str = self.hparams.train_file
        elif state == "val":
            file_str = self.hparams.val_file
        else:
            raise ValueError("state should be train or val")
        
        file_iter = sorted(self.hparams.data_path.glob(file_str), key=lambda x: int(x.name.strip(".json").split("_")[-1]))
        for path in file_iter:
            filename = path.name
            idx = int(filename.strip(".json").split("_")[-1])
            self.create_dataset(path.name, idx, state)

    def create_dataset(self, filename:str, idx:int, state:str):
        cache_file = self.hparams.cache_file.format(state, idx)
        print(f"[INFO] Processing: {filename} | Cache file name: {cache_file}")
        processed_file = self.hparams.ckpt_path / cache_file
        if processed_file.exists():
            print(f"[INFO] cache file already exists! passing the procedure")
            print(f"[INFO] Path: {processed_file}")
            if state == "train":
                self.train_files.append(cache_file)
            elif state == "val":
                self.val_files.append(cache_file)
            else:
                raise ValueError("state should be train or val")
            return None
        else:
            processor = SquadV2Processor()
            if state == "train":
                process_fn = processor.get_train_examples
                is_training = True
                self.train_files.append(cache_file)
            elif state == "val":
                process_fn = processor.get_dev_examples
                is_training = False
                self.val_files.append(cache_file)
            else:
                raise ValueError("state should be train or val")

            examples = process_fn(
                data_dir=self.hparams.data_path, 
                filename=filename
            )

            features = squad_convert_examples_to_features(
                examples=examples,
                tokenizer=self.tokenizer,
                max_seq_length=self.hparams.max_seq_length,
                doc_stride=self.hparams.doc_stride,
                max_query_length=self.hparams.max_query_length,
                is_training=is_training,
                return_dataset=False,
                threads=self.hparams.threads,
            )
            # need to fix all `example_index` and `unique_id` since splitted the dataset only on validation dataset
            self.fix_unique_id(features, state)
            dataset = self.convert_to_tensor(state, features)
            cache = dict(dataset=dataset, examples=examples, features=features)
            torch.save(cache, processed_file)
            print(f"[INFO] cache file saved! {processed_file}")

    def convert_to_tensor(self, state:str, features:List[SquadFeatures]):
        """
        Reference: https://github.com/huggingface/transformers/blob/master/src/transformers/data/processors/squad.py
        Arguments:
            state {str} -- [description]
        """        
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_attention_masks = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
        all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
        
        if state == "train":
            all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
            all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)
            dataset = TensorDataset(
                all_input_ids, all_attention_masks, all_token_type_ids, all_start_positions, all_end_positions
            )
        elif state == "val":
            all_unique_ids = torch.tensor([f.unique_id for f in features], dtype=torch.long)
            dataset = TensorDataset(
                all_input_ids, all_attention_masks, all_token_type_ids, all_unique_ids
            )
        else:
            raise ValueError("state should be train or val")
        return dataset

    def fix_unique_id(self, features:list, state:str="val"):
        if state == "val":
            previous_example_index = -1 
            for fea in tqdm(features, total=len(features), desc="fixing index and ids"):
                fea.unique_id = self.unique_id
                self.unique_id += 1
                
                current_example_index = fea.example_index
                if previous_example_index == current_example_index:
                    fea.example_index = previous_example_index
                else:
                    previous_example_index = fea.example_index
                    fea.example_index = self.example_index
                    self.example_index += 1
        else:
            return None

    def load_cache(self, filename:str, return_dataset:bool=True):
        processed_file = self.hparams.ckpt_path / filename
        cache = torch.load(processed_file)
        dataset, examples, features = cache["dataset"], cache["examples"], cache["features"]

        if return_dataset:
            return dataset
        else:
            return examples, features

    def create_dataloader(self, state:str="train"):
        if state == "train":
            shuffle = True
            batch_size = self.hparams.train_batch_size
            file_list = self.train_files
        elif state == "val":
            shuffle = False
            batch_size = self.hparams.eval_batch_size
            file_list = self.val_files
        else:
            raise ValueError("state should be train or val")

        file_loader = (self.load_cache(filename=file, return_dataset=True) for file in file_list)
        loaders = []
        self.val_dataset_length = []
        for dataset in file_loader: 
            dataloader = DataLoader(
                dataset=dataset,
                batch_size=batch_size,
                shuffle=shuffle,
                num_workers=self.hparams.threads
            )
            loaders.append(dataloader)
            self.val_dataset_length.append(len(dataset))
        return loaders

    def train_dataloader(self):
        return self.create_dataloader(state="train")

    def val_dataloader(self):
        return self.create_dataloader(state="val")

    def forward(self, **kwargs):
        return self.model(**kwargs)

    def training_step(self, batch, batch_idx):
        batch = list(map(torch.cat, zip(*batch)))
        inputs_ids, attention_mask, token_type_ids, start_positions, end_positions = batch

        outputs = self(
            input_ids=inputs_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            start_positions=start_positions,
            end_positions=end_positions
        )

        loss = outputs.loss
        return  {'loss': loss}

    def validation_step(self, batch, batch_idx, dataloader_idx):
        # batch = single dataloader batch not multiple dataloader
        inputs_ids, attention_mask, token_type_ids, data_unique_ids = batch
        outputs = self(
            input_ids=inputs_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            start_positions=None,
            end_positions=None
        )

        # outputs.values: [(B, H), (B, H)] > batch_results: (B, 2, H)
        # B = len(datasets) * batch_size
        batch_results = []
        for i, unique_id in enumerate(data_unique_ids.detach().cpu().tolist()):
            output = [self.tolist(o[i]) for o in outputs.values()]
            start_logits, end_logits = output
            result = SquadResult(unique_id, start_logits, end_logits)
            batch_results.append(result)

        # for i, example_index in enumerate(example_indices):
        #     eval_feature = self.eval_features[example_index.item()]
        #     unique_id = int(eval_feature.unique_id)
        #     output = [self.tolist(o[i]) for o in outputs.values()]
        #     start_logits, end_logits = output
        #     result = SquadResult(unique_id, start_logits, end_logits)
        #     batch_results.append(result)
            
        return batch_results
    
    def train_epoch_end(self, outputs):
        loss = torch.tensor(0, dtype=torch.float)
        for out in outputs:
            loss += out["loss"].detach().cpu()
        loss = loss / len(outputs)

        return {'loss': loss}

    def validation_epoch_end(self, outputs):
        if (self.all_examples == []) or (self.all_features == []):
            for file in self.val_files:
                examples, features = self.load_cache(filename=file, return_dataset=False)
                self.all_examples.extend(examples)
                self.all_features.extend(features)
                del examples
                del features

        all_results = list(flatten(outputs))
        # TODO: See if needed?
        # outputs: [(B, 2, H)] : start_logits, end_logits list
        # B = len(datasets) * batch_size
        # all_results = []
        # for res in outputs:  # res: (B, 2, H)
        #     start_logits, end_logits = res
        #     idx = torch.arange(self.hparams.train_batch_size).repeat(len(self.val_files), 1)  # len(dataset), batch_size
        #     example_idx_to_add = torch.LongTensor([0] + self.val_dataset_length[:-1]).unsqueeze(1)
        #     idx = (idx + example_idx_to_add).view(-1)  # B
        #     for k in idx:
        #         unique_id = self.all_features[k].unique_id
        #         result = SquadResult(unique_id, start_logits, end_logits)
        #     all_results.append(result)
        
        # https://huggingface.co/transformers/_modules/transformers/data/processors/squad.html
        # TODO: Cannot find the key unique_id
        # BUG: must set argument of `trainer: num_sanity_val_steps=0` to avoid error.

        predictions = compute_predictions_logits(
            self.all_examples,
            self.all_features,
            all_results,
            self.hparams.n_best_size,
            self.hparams.max_answer_length,
            self.hparams.do_lower_case,
            self.hparams.ckpt_path / self.hparams.output_prediction_file.format(self.global_step),
            self.hparams.ckpt_path / self.hparams.output_nbest_file.format(self.global_step),
            self.hparams.ckpt_path / self.hparams.output_null_log_odds_file.format(self.global_step),
            self.hparams.verbose_logging,
            self.hparams.version_2_with_negative,
            self.hparams.null_score_diff_threshold,
            self.tokenizer,
        )
        results = squad_evaluate(self.all_examples, predictions)
        accuracy = results["exact"]
        f1 = results["f1"]
        self.log("accuracy", accuracy, on_epoch=True, prog_bar=True)
        self.log("f1", f1, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        t_total = self.total_steps()
        
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 
                "weight_decay": 0.0
            },
        ]
        optimizer = AdamW(
            params=optimizer_grouped_parameters, 
            lr=self.hparams.learning_rate, 
            eps=self.hparams.adam_epsilon
        )
        scheduler = get_linear_schedule_with_warmup(
            optimizer=optimizer, 
            num_warmup_steps=int(t_total * self.hparams.warmup_proportion), 
            num_training_steps=t_total
        )
        
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def total_steps(self):
        r"""
        source: https://github.com/PyTorchLightning/pytorch-lightning/issues/1038
        """
        return len(self.train_dataloader()) * self.hparams.num_train_epochs

In [13]:
def main(args_dict):
    print("[INFO] Using PyTorch Ver", torch.__version__)
    print("[INFO] Seed:", args_dict["random_seed"])
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        filename="epoch{epoch}-f1{f1:.4f}",
        monitor="f1",
        save_top_k=3,
        mode="max",
    )
    pl.seed_everything(args_dict["random_seed"])
    model = Model(**args_dict)
    
    print("[INFO] Start FineTuning")
    trainer = pl.Trainer(
        callbacks=[checkpoint_callback],
        max_epochs=args_dict["num_train_epochs"],
        deterministic=torch.cuda.is_available(),
        gpus=-1 if torch.cuda.is_available() else None,
        num_sanity_val_steps=0
    )
    trainer.fit(model)
    return model

In [14]:
main(args_dict)

Global seed set to 77


[INFO] Using PyTorch Ver 1.6.0
[INFO] Seed: 77


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.weight'

[INFO] Processing: test_train_0.json | Cache file name: test_train_cache_0
[INFO] cache file already exists! passing the procedure
[INFO] Path: /home/simonjisu/code/ckpt/test_train_cache_0
[INFO] Processing: test_train_1.json | Cache file name: test_train_cache_1
[INFO] cache file already exists! passing the procedure
[INFO] Path: /home/simonjisu/code/ckpt/test_train_cache_1
[INFO] Processing: test_val_0.json | Cache file name: test_val_cache_0
[INFO] cache file already exists! passing the procedure
[INFO] Path: /home/simonjisu/code/ckpt/test_val_cache_0
[INFO] Processing: test_val_1.json | Cache file name: test_val_cache_1
[INFO] cache file already exists! passing the procedure
[INFO] Path: /home/simonjisu/code/ckpt/test_val_cache_1
[INFO] Start FineTuning


Model(
  (model): ElectraForQuestionAnswering(
    (electra): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(35000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): ElectraEncoder(
        (layer): ModuleList(
          (0): ElectraLayer(
            (attention): ElectraAttention(
              (self): ElectraSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): ElectraSelfOutput(
                (dense): Linear(in_features=768, out_features

---